# Gemora

In [ ]:
!pip install opendatasets -q
!pip install tensorflow matplotlib seaborn scikit-learn -q


In [ ]:

# import necessary libraries
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import time

import opendatasets as od

In [ ]:
od.download("https://www.kaggle.com/datasets/lsind18/gemstones-images")

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: binojmadhuranga
Your Kaggle Key: ··········
Dataset URL: https://www.kaggle.com/datasets/lsind18/gemstones-images


100%|██████████| 55.2M/55.2M [00:00<00:00, 1.42GB/s]

In [ ]:
import os

base_path = "/content/gemstones-images"
os.listdir(base_path)


['test', 'train']

**Load Training Dataset in TensorFlow**

In [ ]:
!pip install split-folders


In [ ]:
import splitfolders

input_folder = "/content/gemstones-images/train"   # your only folder
output_folder = "/content/gems-split"

splitfolders.ratio(input_folder, output=output_folder, seed=42, ratio=(0.8, 0.1, 0.1))


Copying files: 2856 files [00:00, 4886.83 files/s]


In [ ]:
train_ds = tf.keras.utils.image_dataset_from_directory(
    "/content/gems-split/train",
    image_size=(224, 224),
    batch_size=32
)

val_ds = tf.keras.utils.image_dataset_from_directory(
    "/content/gems-split/val",
    image_size=(224, 224),
    batch_size=32
)

test_ds = tf.keras.utils.image_dataset_from_directory(
    "/content/gems-split/test",
    image_size=(224, 224),
    batch_size=32
)


Found 2247 files belonging to 87 classes.
Found 252 files belonging to 87 classes.
Found 357 files belonging to 87 classes.


In [ ]:
tf.keras.backend.clear_session()


**Build a Transfer Learning Model**

In [ ]:
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras import layers, models
import tensorflow as tf

num_classes = len(train_ds.class_names)

# Base model
base_model = EfficientNetB0(
    weights="imagenet",
    include_top=False,
    input_shape=(224, 224, 3)
)
base_model.trainable = False  # freeze layers first

# Functional model
inputs = tf.keras.Input(shape=(224, 224, 3))
x = base_model(inputs, training=False)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dropout(0.2)(x)
outputs = layers.Dense(num_classes, activation="softmax")(x)

model = tf.keras.Model(inputs, outputs)

model.compile(
    optimizer="adam",
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)

model.summary()


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ efficientnetb0 (Functional)     │ (None, 7, 7, 1280)     │     4,049,571 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 1280)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 87)             │       111,447 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,161,018 (15.87 MB)

 Trainable params: 111,447 (435.34 KB)

 Non-trainable params: 4,049,571 (15.45 MB)

**Train the model**

In [ ]:
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=15
)


Epoch 1/15
71/71 ━━━━━━━━━━━━━━━━━━━━ 201s 3s/step - accuracy: 0.0879 - loss: 4.1800 - val_accuracy: 0.4643 - val_loss: 2.7267
Epoch 2/15
71/71 ━━━━━━━━━━━━━━━━━━━━ 184s 3s/step - accuracy: 0.5523 - loss: 2.3347 - val_accuracy: 0.5833 - val_loss: 1.9702
Epoch 3/15
71/71 ━━━━━━━━━━━━━━━━━━━━ 201s 3s/step - accuracy: 0.7131 - loss: 1.6166 - val_accuracy: 0.6389 - val_loss: 1.6172
Epoch 4/15
71/71 ━━━━━━━━━━━━━━━━━━━━ 198s 3s/step - accuracy: 0.7796 - loss: 1.2417 - val_accuracy: 0.6587 - val_loss: 1.4221
Epoch 5/15
71/71 ━━━━━━━━━━━━━━━━━━━━ 181s 3s/step - accuracy: 0.8140 - loss: 1.0104 - val_accuracy: 0.6865 - val_loss: 1.2823
Epoch 6/15
71/71 ━━━━━━━━━━━━━━━━━━━━ 204s 3s/step - accuracy: 0.8602 - loss: 0.8336 - val_accuracy: 0.7024 - val_loss: 1.2061
Epoch 7/15
71/71 ━━━━━━━━━━━━━━━━━━━━ 187s 3s/step - accuracy: 0.8732 - loss: 0.7194 - val_accuracy: 0.6984 - val_loss: 1.1273
Epoch 8/15
71/71 ━━━━━━━━━━━━━━━━━━━━ 200s 3s/step - accuracy: 0.9027 - loss: 0.6072 - val_accuracy: 0.7222 - v

**Fine tune With Augmentation**

In [ ]:
data_augmentation = tf.keras.Sequential([
    tf.keras.layers.RandomFlip("horizontal"),
    tf.keras.layers.RandomRotation(0.1),
    tf.keras.layers.RandomZoom(0.1),
    tf.keras.layers.RandomBrightness(0.1),
])


In [ ]:
inputs = tf.keras.Input(shape=(224,224,3))
x = data_augmentation(inputs)
x = base_model(x, training=False)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dropout(0.2)(x)
outputs = layers.Dense(num_classes, activation="softmax")(x)

model = tf.keras.Model(inputs, outputs)


In [ ]:
model.compile(
    optimizer="adam",
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)

history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=15
)


Epoch 1/15
71/71 ━━━━━━━━━━━━━━━━━━━━ 287s 4s/step - accuracy: 0.2418 - loss: 3.3811 - val_accuracy: 0.6667 - val_loss: 1.1553
Epoch 2/15
71/71 ━━━━━━━━━━━━━━━━━━━━ 254s 4s/step - accuracy: 0.7152 - loss: 0.9672 - val_accuracy: 0.7460 - val_loss: 0.9020
Epoch 3/15
71/71 ━━━━━━━━━━━━━━━━━━━━ 257s 4s/step - accuracy: 0.8426 - loss: 0.5118 - val_accuracy: 0.7183 - val_loss: 0.9405
Epoch 4/15
71/71 ━━━━━━━━━━━━━━━━━━━━ 257s 4s/step - accuracy: 0.8981 - loss: 0.3642 - val_accuracy: 0.7143 - val_loss: 0.8879
Epoch 5/15
71/71 ━━━━━━━━━━━━━━━━━━━━ 266s 4s/step - accuracy: 0.9270 - loss: 0.2819 - val_accuracy: 0.7302 - val_loss: 0.9679
Epoch 6/15
71/71 ━━━━━━━━━━━━━━━━━━━━ 257s 4s/step - accuracy: 0.9302 - loss: 0.2234 - val_accuracy: 0.7381 - val_loss: 0.8951
Epoch 7/15
71/71 ━━━━━━━━━━━━━━━━━━━━ 267s 4s/step - accuracy: 0.9486 - loss: 0.1739 - val_accuracy: 0.6825 - val_loss: 1.0370
Epoch 8/15
71/71 ━━━━━━━━━━━━━━━━━━━━ 262s 4s/step - accuracy: 0.9506 - loss: 0.1554 - val_accuracy: 0.7063 - v

**SAVE**

In [ ]:
import json

with open("labels.json", "w") as f:
    json.dump(train_ds.class_names, f, indent=2)


In [ ]:
model.save("gem_identifier_model_v3.keras")


In [ ]:
#This gem_identifier_model_v3.keras model is Current best performing model.

**2nd Fine Tune**

In [ ]:
data_augmentation = tf.keras.Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.2),
    layers.RandomZoom(0.2),
    layers.RandomContrast(0.2),
])


In [ ]:
for layer in base_model.layers[:-30]:
    layer.trainable = False


In [ ]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(3e-6),  # LOWER LR
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)



NameError: name 'model' is not defined

In [ ]:
history_2 = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=8
)


Epoch 1/8
72/72 ━━━━━━━━━━━━━━━━━━━━ 147s 2s/step - accuracy: 0.8162 - loss: 0.9285 - val_accuracy: 0.7250 - val_loss: 0.9739
Epoch 2/8
72/72 ━━━━━━━━━━━━━━━━━━━━ 150s 2s/step - accuracy: 0.8373 - loss: 0.8757 - val_accuracy: 0.7023 - val_loss: 1.0663
Epoch 3/8
72/72 ━━━━━━━━━━━━━━━━━━━━ 150s 2s/step - accuracy: 0.8356 - loss: 0.8474 - val_accuracy: 0.6830 - val_loss: 1.1457
Epoch 4/8
72/72 ━━━━━━━━━━━━━━━━━━━━ 131s 2s/step - accuracy: 0.8553 - loss: 0.7955 - val_accuracy: 0.6708 - val_loss: 1.1964
Epoch 5/8
72/72 ━━━━━━━━━━━━━━━━━━━━ 160s 2s/step - accuracy: 0.8522 - loss: 0.8006 - val_accuracy: 0.6620 - val_loss: 1.2206
Epoch 6/8
72/72 ━━━━━━━━━━━━━━━━━━━━ 131s 2s/step - accuracy: 0.8656 - loss: 0.7800 - val_accuracy: 0.6585 - val_loss: 1.2269
Epoch 7/8
72/72 ━━━━━━━━━━━━━━━━━━━━ 140s 2s/step - accuracy: 0.8539 - loss: 0.7852 - val_accuracy: 0.6602 - val_loss: 1.2214
Epoch 8/8
72/72 ━━━━━━━━━━━━━━━━━━━━ 154s 2s/step - accuracy: 0.8636 - loss: 0.7413 - val_accuracy: 0.6620 - val_loss:

In [ ]:
model.save("gem_identifier_model.keras_v4")